1.6 Exam vB, PROBLEM 1
Maximum Points = 14
In this problem you will do rejection sampling from complicated distributions, you will also be using
your samples to compute certain integrals, a method known as Monte Carlo integration: (Keep in
mind that choosing a good sampling distribution is often key to avoid too much rejection)
1. [4p] Fill in the remaining part of the function problem1_rejection in order to produce samples
from the below distribution using rejection sampling: (Hint: F is the distribution function)
$$F[x] = 0, x ≤ 0,\frac{e^{x^2} - x^2 - 1}{e -2}, 0 < x < 1, 1, x ≥ 1$$
2. [2p] Produce 100000 samples (use fewer if it takes too long (more than 10 sec) and
you cannot find a solution) and put the answer in problem1_samples from the above
distribution and plot the histogram together with the true density.
3. [2p] Use the above 100000 samples (problem1_samples) to approximately compute the integral
$$\int^1_0 sin(x) \frac{2(e^{x^2}-1)x}{e - 2}dx$$
and store the result in problem1_integral.
4. [2p] Use Hoeffdings inequality to produce a 95% confidence interval of the integral above and
store the result as a tuple in the variable problem1_interval
5. [4p] Fill in the remaining part of the function problem1_rejection_2 in order to produce
samples from the below distribution using rejection sampling:
$$F[x] = 0, x ≤ 0, 20xe^{20−1/x}, 0 < x < 1/20 1, x ≥1/20$$
Hint: this is tricky because if you choose the wrong sampling distribution you reject at least
9 times out of 10. You will get points based on how long your code takes to create a certain
number of samples, if you choose the correct sampling distribution you can easily create 100000
samples within 2 seconds.


In [ ]:
# Part 1
# Distribution from part 1
# write the code in this function to produce samples from the distribution
# in the assignment
# Make sure you choose a good sampling distribution to avoid unnecessary
# rejections
# Return a numpy array of length n_samples
import numpy as np

def problem1_rejection(n_samples=1):
    samples = []
    M = 2 * (np.e - 1) / (np.e - 2)  # safe upper bound

    while len(samples) < n_samples:
        x = np.random.rand()
        u = np.random.rand()
        fx = (2 * x * (np.exp(x**2) - 1)) / (np.e - 2)
        if u <= fx / M:
            samples.append(x)

    return np.array(samples)


In [ ]:
# Part 2
import matplotlib.pyplot as plt

problem1_samples = problem1_rejection(100000)

# True density
x_grid = np.linspace(0, 1, 500)
true_density = (2 * x_grid * (np.exp(x_grid**2) - 1)) / (np.e - 2)

plt.figure()
plt.hist(problem1_samples, bins=50, density=True, alpha=0.6)
plt.plot(x_grid, true_density)
plt.xlabel("x")
plt.ylabel("Density")
plt.show()


In [ ]:
# Part 3
problem1_integral = np.mean(np.sin(problem1_samples))


In [ ]:
# Part 4
n = len(problem1_samples)
epsilon = np.sqrt(np.log(2 / 0.05) / (2 * n))

problem1_interval = (
    problem1_integral - epsilon,
    problem1_integral + epsilon
)



In [ ]:
# Part 5

#Let 𝑌 = 1/X, then Y has exponential like tail so we sample Y from Exp(1)


# Distribution from part 2
# write the code in this function to produce samples from the distribution␣
# in the assignment
# Make sure you choose a good sampling distribution to avoid unnecessary
# rejections
    # Return a numpy array of length n_samples
def problem1_rejection_2(n_samples=1):
    samples = []

    while len(samples) < n_samples:
        y = np.random.exponential() + 20  # shift
        x = 1 / y

        if x < 1 / 20:
            u = np.random.rand()
            fx = 20 * np.exp(20 - 1/x) / x**2
            gy = np.exp(-(y - 20))         # proposal density
            M = 20 * np.exp(20)

            if u <= fx / (M * gy):
                samples.append(x)

    return np.array(samples)



1.7 Exam vB, PROBLEM 2
Maximum Points = 14
In this problem we have data consisting of user behavior on a website. The pages of the website are
just numbers in the dataset 0, 1, 2, . . . and each row consists of a user, a source and a destination
page. This signifies that the user was on the source page and clicked a link leading them to the
destination page. The goal is to improve the user experience by decreasing load time of the next
page visited, as such we need a good estimate for the next site likely to be visited. We will model
this using a homogeneous Markov chain, each row in the data-file then corresponds to a single
realization of a transition.
1. [3p] Load the data in the file data/websites.csv and construct a matrix of size n_pages x n_pages which is the maximum likelihood estimate of the true transition matrix for the
Markov chain. Here the ordering of the states are exactly the ones in the data-file, that is
page 0 has index 0 in the matrix.
2. [4p] A page loads in Exp(3) (Exponentially distributed with mean 1/3) seconds if not preloaded
and loads with Exp(20) (Exponentially distributed with mean 1/20) seconds if preloaded and
we only preload the most likely next site. Given that we start in page 8 simulate 10000 load
times from page 1 (that is, only a single step), store the result in the variable indicated in
the cell. Repeat the experiment but this time preload the two most likely pages and store the
result in the indicated variable.
3. [3p] Compare the average (empirical) load time from part 2 with the theoretical one of no
pre-loading. Does the load time improve, how did you come to this conclusion? (Explain in
the free text field).
4. [4p] Calculate the stationary distribution of the Markov chain and calculate the expected load
time with respect to the stationary distribution.


In [ ]:
# Part 1: 3 points
# Load the data from the file data/websites.csv and estimate the transition
# matrix of the Markov chain
# Store the estimated transition matrix in the variable
# problem2_transition_matrix below
import numpy as np
import pandas as pd

# Load data
df = pd.read_csv("data/websites.csv")

# Assume columns are named as follows (typical):
# user_id, source, destination
sources = df.iloc[:, 1].values
destinations = df.iloc[:, 2].values

# Number of states
problem2_n_states = int(max(sources.max(), destinations.max())) + 1

# Count transitions
counts = np.zeros((problem2_n_states, problem2_n_states))

for s, d in zip(sources, destinations):
    counts[s, d] += 1

# Normalize rows to get transition probabilities
problem2_transition_matrix = counts / counts.sum(axis=1, keepdims=True)
 # A numpy array of shape (problem2_n_states,
# problem2_n_states)
# Store the number of states in the variable problem2_n_states below
# An integer

In [ ]:
def simulate_load_times(start_page, P, preload_pages, n=10000):
    load_times = []

    for _ in range(n):
        next_page = np.random.choice(len(P), p=P[start_page])

        if next_page in preload_pages:
            load_times.append(np.random.exponential(scale=1/20))
        else:
            load_times.append(np.random.exponential(scale=1/3))

    return np.array(load_times)


# Part 2: 4 points
# Simulate the website load times for the next page of 10000 users that are
# currently on page 8 (recall indexing starts at 0) when we only preload the
# most likely page.
# Store the simulated page load times in the variable
# problem2_page_load_times_top below
start_page = 8
row = problem2_transition_matrix[start_page]

top_page = np.argmax(row)

problem2_page_load_times_top = simulate_load_times(
    start_page,
    problem2_transition_matrix,
    preload_pages={top_page},
    n=10000
)
 # A numpy array of shape (10000,)
# Repeat the simulation of load times for the next page of 10000 users that are
# currently on page 8 when we preload the two most likely pages.
# Store the simulated page load times in the variable
# problem2_page_load_times_two below
top_two_pages = set(np.argsort(row)[-2:])

problem2_page_load_times_two = simulate_load_times(
    start_page,
    problem2_transition_matrix,
    preload_pages=top_two_pages,
    n=10000
)
 # A numpy array of shape (10000,)


In [ ]:
# Part 3: 3 points
# Calculate the true expected load time for loading a page without pre-loading
# the next page and store it in the variable below
problem2_avg = 1/3
 # A float
# Is the average load time for loading a page without pre-loading the next page
# larger than the average load time for loading a page after pre-loading the
# next most likely page?
problem2_comparison = (
    problem2_page_load_times_top.mean() < problem2_avg
) # True / False

1.8 Free text answer
Put the explanation for part 3 of how you made the decision about problem2_comparison below
this line in this cell. In order to enter edit mode you can doubleclick this cell or select it and press
enter.

The expected load time without preloading is 1/3 seconds since page loads follow an Exp(3) distribution. I compared this theoretical value with the empirical average load time obtained when preloading the most likely next page. Since the empirical average load time is smaller than 1/3, preloading improves the load time. This conclusion is based on comparing the expected values rather than individual samples.

In [ ]:
# Part 4: 4 points
# Begin by calculating the stationary distribution of the Markov chain and store it in the variable below
# WARNING: Since the transition matrix is not symmetric, numpy might make the output of the eigenvectors complex, you can use np.real() to get the real part of the eigenvectors
# Store the stationary distribution in the variable below called problem2_stationary_distribution
# Eigenvector of P^T corresponding to eigenvalue 1
eigvals, eigvecs = np.linalg.eig(problem2_transition_matrix.T)

idx = np.argmin(np.abs(eigvals - 1))
stationary = np.real(eigvecs[:, idx])
stationary = stationary / stationary.sum()

problem2_stationary_distribution = stationary
 # A numpy array of shape (problem2_n_states,)
# Now use the above stationary distribution to calculate the average load time
# for loading a page after pre-loading the next most likely page according to
# the stationary distribution
# Store the average load time in the variable below
expected_load_times = []

for i in range(problem2_n_states):
    p_max = problem2_transition_matrix[i].max()
    expected_time = p_max * (1/20) + (1 - p_max) * (1/3)
    expected_load_times.append(expected_time)

expected_load_times = np.array(expected_load_times)

problem2_avg_stationary = np.dot(
    problem2_stationary_distribution,
    expected_load_times
)
 # A float


1.9 Exam vB, PROBLEM 3
Maximum Points = 12
In this problem we are interested in fraud detection in an e-commerce system. In this problem we
are given the outputs of a classifier that predicts the probabilities of fraud, your goal is to explore
the threshold choice as in individual assignment 4. The costs associated with the predictions are:

• True Positive (TP): Detecting fraud and blocking the transaction costs the company 100
(manual review etc.)

• True Negative (TN): Allowing a legitimate transaction has no cost.

• False Positive (FP): Incorrectly classifying a legitimate transaction as fraudulent costs 120
(customer dissatisfaction plus operational expenses for reversing the decision).

• False Negative (FN): Missing a fraudulent transaction costs the company 600 (e.g., fraud loss plus potential reputational damage or penalties).

The code cells contain more detailed instructions, THE FIRST CODE CELL INITIALIZES YOUR VARIABLES

1. [3p] Complete filling the function cost to compute the average cost of a prediction model
under a certain prediction threshold. Plot the cost as a function of the threshold (using the
validation data provided in the first code cell of this problem), between 0 and 1 with 0.01
increments.
2. [2.5p] Find the threshold that minimizes the cost and calculate the cost at that threshold on
the validation data. Also calculate the precision and recall at the optimal threshold on the
validation data on class 1 and 0.
3. [2.5p] Repeat step 2, but this time find the best threshold to minimize the 0−1 loss. Calculate
the difference in cost between the threshold found in part 2 with the one just found in part 3.
4. [4p] Provide a confidence interval around the optimal cost (with 95% confidence) applied to
the test data and explain all the assumption you made.


In [ ]:
# RUN THIS CELL TO GET THE DATA
# We start by loading the data
import pandas as pd
PROBLEM3_DF = pd.read_csv('data/fraud.csv')
Y = PROBLEM3_DF['Class'].values
X = PROBLEM3_DF[['V%d' % i for i in range(1,5)]+['Amount']].values
# We will split the data into training, testing and validation sets
from Utils import train_test_validation
PROBLEM3_X_train, PROBLEM3_X_test, PROBLEM3_X_val, PROBLEM3_y_train,␣
,→PROBLEM3_y_test, PROBLEM3_y_val =␣
,→train_test_validation(X,Y,shuffle=True,random_state=1)
# From this we will train a logistic regression model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(PROBLEM3_X_train,PROBLEM3_y_train)
# THE FOLLOWING CODE WILL PRODUCE THE ARRAYS YOU NEED FOR THE PROBLEM
PROBLEM3_y_pred_proba_val = lr.predict_proba(PROBLEM3_X_val)[:,1]
PROBLEM3_y_true_val = PROBLEM3_y_val
PROBLEM3_y_pred_proba_test = lr.predict_proba(PROBLEM3_X_test)[:,1]
PROBLEM3_y_true_test = PROBLEM3_y_test

In [ ]:
# Part 1: 3 points
# Implement the following function that calculates the cost of a binary
# classifier according to
# the specification in the problem statement
# See the comments inside the function for details of the parameters
#def cost(y_true,y_predict_proba,threshold):
# y_true is a numpy array of shape (n_samples,) with binary labels
# y_predict_proba is a numpy array of shape (n_samples,) with predicted
# probabilities
# threshold is a float between 0 and 1
# When returning the cost, you should return the average cost per sample
# thus it should be a value
#    return XXX # A float
# Provide the code below to plot the cost as a function of the threshold
# using the validation data, specifically the arrays PROBLEM3_y_true_val and
# PROBLEM3_y_pred_proba_val.
# The plot should be between 0 and 1 with 0.01 increments
# The y-axis should be the cost and the x-axis should be the threshold

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score

def cost(y_true, y_predict_proba, threshold):
    """
    Compute average cost per sample for a classifier under a given threshold.

    TP cost = 100
    TN cost = 0
    FP cost = 120
    FN cost = 600
    """
    y_pred = (y_predict_proba >= threshold).astype(int)
    
    # True positives, false positives, true negatives, false negatives
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    total_cost = TP*100 + TN*0 + FP*120 + FN*600
    avg_cost = total_cost / len(y_true)
    
    return avg_cost

thresholds = np.arange(0, 1.01, 0.01)
costs = [cost(PROBLEM3_y_true_val, PROBLEM3_y_pred_proba_val, t) for t in thresholds]

plt.figure()
plt.plot(thresholds, costs)
plt.xlabel("Threshold")
plt.ylabel("Average cost per sample")
plt.title("Cost vs Threshold on validation set")
plt.show()




In [ ]:
# Part 2: 2.5 points
# Use the cost function you just implemented above to find the threshold that minimizes the cost
# using the validation data, specifically the arrays PROBLEM3_y_true_val and PROBLEM3_y_pred_proba_val.
# Store the threshold in the variable below
problem3_threshold = XXX # A float between 0 and 1
# Now calculate the cost of the classifier using the validation data and the
# threshold you just found
# using the validation data, specifically the arrays PROBLEM3_y_true_val and
# PROBLEM3_y_pred_proba_val.
# Store the cost in the variable below
problem3_cost_val = XXX # A float
# Using the threshold you just found, calculate the predicted labels of the
# classifier on the validation data
# put the predicted labels in the variable below
problem3_y_pred_val = XXX # A numpy array of shape (n_samples,) with values 0 or 1
# Calculate the precision and recall of the classifier of class 1 using the
# threshold you just found
# using the validation data, specifically the arrays PROBLEM3_y_true_val and
# PROBLEM3_y_pred_proba_val.
problem3_precision_1 = XXX # A float between 0 and 1
problem3_recall_1 = XXX # A float between 0 and 1
# Calculate the precision and recall of the classifier of class 0 using the
# threshold you just found
# using the validation data, specifically the arrays PROBLEM3_y_true_val and
# PROBLEM3_y_pred_proba_val.
problem3_precision_0 = XXX # A float between 0 and 1
problem3_recall_0 = XXX # A float between 0 and 1

# Optimal threshold
problem3_threshold = thresholds[np.argmin(costs)]

# Cost at optimal threshold
problem3_cost_val = cost(PROBLEM3_y_true_val, PROBLEM3_y_pred_proba_val, problem3_threshold)

# Predicted labels at optimal threshold
problem3_y_pred_val = (PROBLEM3_y_pred_proba_val >= problem3_threshold).astype(int)

# Precision and recall for class 1
problem3_precision_1 = precision_score(PROBLEM3_y_true_val, problem3_y_pred_val, pos_label=1)
problem3_recall_1 = recall_score(PROBLEM3_y_true_val, problem3_y_pred_val, pos_label=1)

# Precision and recall for class 0
problem3_precision_0 = precision_score(PROBLEM3_y_true_val, problem3_y_pred_val, pos_label=0)
problem3_recall_0 = recall_score(PROBLEM3_y_true_val, problem3_y_pred_val, pos_label=0)


In [ ]:
 # Part 3: 2.5 points
# Find the threshold that minimizes the $0-1$ loss using the validation data
# specifically the arrays PROBLEM3_y_true_val and PROBLEM3_y_pred_proba_val.
# Store the threshold in the variable below
problem3_threshold_01 = XXX # A float between 0 and 1
# Now calculate the difference in cost (using the cost function you implemented
# in step 1) between the optimal one chosen in part 2 and the one chosen in part 3
# by taking the cost with the threshold found in part 3 and subtracting the
# cost with the threshold found in part 2 to get a positive value
problem3_cost_difference = XXX # A float

# 0-1 loss = classification error
errors = [np.mean((PROBLEM3_y_true_val != (PROBLEM3_y_pred_proba_val >= t).astype(int))) for t in thresholds]
problem3_threshold_01 = thresholds[np.argmin(errors)]

# Difference in cost between optimal threshold and 0-1 loss threshold
problem3_cost_difference = abs(
    cost(PROBLEM3_y_true_val, PROBLEM3_y_pred_proba_val, problem3_threshold_01)
    - problem3_cost_val
)



In [ ]:
# Part 4: 4 points
# Using the threshold problem3_threshold use Hoeffdings inequality to provide a
# confidence interval
# for the cost of the classifier with 95 % confidence using the test data.
# Specifically the arrays PROBLEM3_y_true_test and PROBLEM3_y_pred_proba_test.
# Store the lower and upper bounds of the confidence interval in the variables
# below
n_test = len(PROBLEM3_y_true_test)
C_hat = cost(PROBLEM3_y_true_test, PROBLEM3_y_pred_proba_test, problem3_threshold)

epsilon = np.sqrt(np.log(2/0.05) / (2*n_test))

problem3_lower_bound = C_hat - epsilon
problem3_upper_bound = C_hat + epsilon


1.10 Free text answer
Put your explanation for part 4 below this line in this cell. Doubleclick to enter edit mode as before.

I assumed that the individual sample costs are independent and bounded between 0 and the maximum cost (here 600 for a false negative). Using Hoeffding’s inequality, I constructed a 95% confidence interval around the empirical mean cost on the test set. This gives a conservative, distribution-free bound on the true expected cost per transaction. The interval represents where we expect the true average cost to lie with 95% confidence, assuming the test samples are independent realizations from the same distribution.